In [2]:
import pandas as pd, numpy as np
from shapely.geometry import Polygon
from shapely.geometry import Point
import geopandas
import matplotlib.pyplot as plot

In [7]:
output = 'Output/' # select folder with data in it

#specify datatypes so FIPS numbers, etc. read in properly
dtypes = {'Geo_STATE': 'str', 'Geo_FIPS': 'str', 'Geo_COUNTY': 'str', 'h_tract': 'str', 'FIPS_County': 'str'}

tract_data = pd.read_csv(output + 'tract_data.csv', delimiter=',', dtype=dtypes)

print(len(tract_data))

tract_data.head(2)

8057


,Unnamed: 0,Geo_FIPS,alljobs,ltbajobs,lt40kjobs,county_labor,county_unemp,county_med_rent,county_med_value,CBSA Code,...,bach_degree,pop_over_25,hh_kids,total_hh,med_rent,med_housevalue,unemp_civ,civ_labor_force,area_sqmi,HU_density
0,0,06001400100,763651.0,358749.0,301972.0,862353,61327,1432,593500,41860.0,...,1981.0,2478.0,241.0,1292.0,3202.0,1074100.0,75,1643,2.657,486.262695
1,1,06001400200,948962.0,453334.0,376760.0,862353,61327,1432,593500,41860.0,...,1278.0,1559.0,184.0,813.0,1770.0,978900.0,41,1270,0.230,3534.782471


In [14]:
#specify datatypes so FIPS numbers, etc. read in properly
dtypes2 = {'GEOID': 'str', 'Agency': 'str', 'Stop ID': 'str', 'h_tract': 'str', 'FIPS_County': 'str'}

station_data = pd.read_csv(output + 'best_matches.csv', delimiter=',', dtype=dtypes2)

station_small = station_data[['GEOID','Agency','Stop ID']]
                              #,'station_area','tract_area','geometry','overlap_area']]
station_small['neartransit'] = 1

print(len(station_small))

station_small.head(2)

25862


C:\Users\jared\Miniconda3\envs\analysis\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,GEOID,Agency,Stop ID,neartransit
0,06043000400,Yosemite Valley Shuttle System,766882,1
1,06071009910,Victor Valley Transit Authority,813390,1


merge together datasets by census tract; we want to keep all census tracts so we can calculate averages for tracts near transit and those not near transit

In [18]:
pd.merge(tract_data, station_small, right_on="GEOID", left_on="Geo_FIPS", how="left")

values = {'neartransit': 0}
merge_data = merge_data.fillna(value=values)

print(len(merge_data))

merge_data.head(3)

31613


,Unnamed: 0,Geo_FIPS,alljobs,ltbajobs,lt40kjobs,county_labor,county_unemp,county_med_rent,county_med_value,CBSA Code,...,med_rent,med_housevalue,unemp_civ,civ_labor_force,area_sqmi,HU_density,GEOID,Agency,Stop ID,neartransit
0,0,06001400100,763651.0,358749.0,301972.0,862353,61327,1432,593500,41860.0,...,3202.0,1074100.0,75,1643,2.657,486.262695,NaN,NaN,NaN,0.0
1,1,06001400200,948962.0,453334.0,376760.0,862353,61327,1432,593500,41860.0,...,1770.0,978900.0,41,1270,0.230,3534.782471,06001400200,AC Transit,9903430,1.0
2,1,06001400200,948962.0,453334.0,376760.0,862353,61327,1432,593500,41860.0,...,1770.0,978900.0,41,1270,0.230,3534.782471,06001400200,AC Transit,9903420,1.0


In [23]:
unique_tract00 = merge_data.drop_duplicates(subset='Geo_FIPS')
len(unique_tract)

8057

In [31]:
cols = ['renters', 'occ_HU', 'hispanic', 'black',
       'white', 'asian', 'nhpi', 'total_pop', 'below200pov',
       'total_pov_status', 'rent_occ_HU', 'low_inc_rent_burden',
       'white_pov_tot', 'white_pov', 'black_pov_tot', 'black_pov',
       'asian_pov_tot', 'asian_pov', 'nhpi_pov_tot', 'nhpi_pov',
       'hispanic_pov_tot', 'hispanic_pov', 'sfdetach', 'smallmf', 'medmf',
       'bigmf', 'total_hu', 'vacant_hu', 'total_hu2', 'since2000',
       'before1950', 'total_structure', 'bach_degree', 'pop_over_25',
       'hh_kids', 'total_hh','unemp_civ','civ_labor_force','area_sqmi']

avg_cols = ['alljobs','med_rent']

unique_tract = unique_tract00[unique_tract00['neartransit'] == 1]
print(len(unique_tract))

qtracts_tots=unique_tract[cols].groupby(unique_tract['CBSA Title']).sum()
qtracts_avgs=unique_tract[avg_cols].groupby(unique_tract['CBSA Title']).mean()


qtracts_stats = pd.DataFrame(qtracts_tots.iloc[:,7])

qtracts_stats['pct_rent']=qtracts_tots['renters'] / qtracts_tots['occ_HU']

qtracts_stats['pct_white']=qtracts_tots['white'] / qtracts_tots['total_pop']
qtracts_stats['pct_hispanic']=qtracts_tots['hispanic'] / qtracts_tots['total_pop']
qtracts_stats['pct_black']=qtracts_tots['black'] / qtracts_tots['total_pop']
qtracts_stats['pct_asian']=qtracts_tots['asian'] / qtracts_tots['total_pop']

qtracts_stats['pct_below200pov']=qtracts_tots['below200pov'] / qtracts_tots['total_pov_status']

qtracts_stats['hispanic_pov']=qtracts_tots['hispanic_pov'] / qtracts_tots['hispanic_pov_tot']
qtracts_stats['black_pov']=qtracts_tots['black_pov'] / qtracts_tots['black_pov_tot']
qtracts_stats['asian_pov']=qtracts_tots['asian_pov'] / qtracts_tots['asian_pov_tot']
qtracts_stats['white_pov']=qtracts_tots['white_pov'] / qtracts_tots['white_pov_tot']

qtracts_stats['pct_sfdetach']=qtracts_tots['sfdetach'] / qtracts_tots['total_hu']
qtracts_stats['pct_smallmf']=qtracts_tots['smallmf'] / qtracts_tots['total_hu']
qtracts_stats['pct_bigmf']=qtracts_tots['bigmf'] / qtracts_tots['total_hu']
qtracts_stats['pct_medmf']=qtracts_tots['medmf'] / qtracts_tots['total_hu']
qtracts_stats['pct_vacant']=qtracts_tots['vacant_hu'] / qtracts_tots['total_hu2']

qtracts_stats['pct_since2000']=qtracts_tots['since2000'] / qtracts_tots['total_structure']
qtracts_stats['pct_before1950']=qtracts_tots['before1950'] / qtracts_tots['total_structure']

qtracts_stats['pct_bach_degree']=qtracts_tots['bach_degree'] / qtracts_tots['pop_over_25']
qtracts_stats['pct_hh_kids']=qtracts_tots['hh_kids'] / qtracts_tots['total_hh']
qtracts_stats['pct_unemp_civ']=qtracts_tots['unemp_civ'] / qtracts_tots['civ_labor_force']
qtracts_stats['density']=qtracts_tots['total_pop'] / qtracts_tots['area_sqmi']

qtracts_stats['alljobs']=qtracts_avgs['alljobs']
qtracts_stats['med_rent']=qtracts_avgs['med_rent']

qtracts_stats.to_csv(output + 'tract_stats_neartransit.csv')

qtracts_stats.head()

2306


,total_pop,pct_rent,pct_white,pct_hispanic,pct_black,pct_asian,pct_below200pov,hispanic_pov,black_pov,asian_pov,...,pct_medmf,pct_vacant,pct_since2000,pct_before1950,pct_bach_degree,pct_hh_kids,pct_unemp_civ,density,alljobs,med_rent
CBSA Title,,,,,,,,,,,,,,,,,,,,,
"Bakersfield, CA",91716.0,0.529941,0.296306,0.607789,0.049381,0.020978,0.564073,0.313048,0.337901,0.179255,...,0.088919,0.060671,0.176925,0.116991,0.080930,0.487131,0.135765,3400.919632,1.622275e+05,965.428571
"Fresno, CA",161117.0,0.634228,0.263455,0.519883,0.091573,0.097519,0.617283,0.424780,0.499259,0.370059,...,0.194364,0.076157,0.086132,0.187395,0.170654,0.391932,0.140613,5680.734789,2.297754e+05,849.818182
"Los Angeles-Long Beach-Anaheim, CA",5532219.0,0.655263,0.224922,0.527070,0.091684,0.129436,0.461594,0.260321,0.254984,0.160252,...,0.198354,0.061286,0.072423,0.304578,0.284188,0.334971,0.093325,8607.577251,1.251731e+06,1304.005622
"Merced, CA",17792.0,0.537338,0.380789,0.374157,0.067558,0.141918,0.443744,0.324188,0.144863,0.136275,...,0.066066,0.075075,0.379213,0.093594,0.277091,0.373918,0.184378,2327.577150,5.304400e+04,1014.500000
"Modesto, CA",7600.0,0.695426,0.530395,0.312237,0.044211,0.058816,0.403111,0.224810,0.380054,0.134091,...,0.147416,0.067291,0.004509,0.183143,0.179587,0.363704,0.133315,4051.172607,1.460130e+05,856.500000


In [32]:
unique_tract = unique_tract00[unique_tract00['neartransit'] == 0]
print(len(unique_tract))

qtracts_tots=unique_tract[cols].groupby(unique_tract['CBSA Title']).sum()
qtracts_avgs=unique_tract[avg_cols].groupby(unique_tract['CBSA Title']).mean()


qtracts_stats = pd.DataFrame(qtracts_tots.iloc[:,7])

qtracts_stats['pct_rent']=qtracts_tots['renters'] / qtracts_tots['occ_HU']

qtracts_stats['pct_white']=qtracts_tots['white'] / qtracts_tots['total_pop']
qtracts_stats['pct_hispanic']=qtracts_tots['hispanic'] / qtracts_tots['total_pop']
qtracts_stats['pct_black']=qtracts_tots['black'] / qtracts_tots['total_pop']
qtracts_stats['pct_asian']=qtracts_tots['asian'] / qtracts_tots['total_pop']

qtracts_stats['pct_below200pov']=qtracts_tots['below200pov'] / qtracts_tots['total_pov_status']

qtracts_stats['hispanic_pov']=qtracts_tots['hispanic_pov'] / qtracts_tots['hispanic_pov_tot']
qtracts_stats['black_pov']=qtracts_tots['black_pov'] / qtracts_tots['black_pov_tot']
qtracts_stats['asian_pov']=qtracts_tots['asian_pov'] / qtracts_tots['asian_pov_tot']
qtracts_stats['white_pov']=qtracts_tots['white_pov'] / qtracts_tots['white_pov_tot']

qtracts_stats['pct_sfdetach']=qtracts_tots['sfdetach'] / qtracts_tots['total_hu']
qtracts_stats['pct_smallmf']=qtracts_tots['smallmf'] / qtracts_tots['total_hu']
qtracts_stats['pct_bigmf']=qtracts_tots['bigmf'] / qtracts_tots['total_hu']
qtracts_stats['pct_medmf']=qtracts_tots['medmf'] / qtracts_tots['total_hu']
qtracts_stats['pct_vacant']=qtracts_tots['vacant_hu'] / qtracts_tots['total_hu2']

qtracts_stats['pct_since2000']=qtracts_tots['since2000'] / qtracts_tots['total_structure']
qtracts_stats['pct_before1950']=qtracts_tots['before1950'] / qtracts_tots['total_structure']

qtracts_stats['pct_bach_degree']=qtracts_tots['bach_degree'] / qtracts_tots['pop_over_25']
qtracts_stats['pct_hh_kids']=qtracts_tots['hh_kids'] / qtracts_tots['total_hh']
qtracts_stats['pct_unemp_civ']=qtracts_tots['unemp_civ'] / qtracts_tots['civ_labor_force']
qtracts_stats['density']=qtracts_tots['total_pop'] / qtracts_tots['area_sqmi']

qtracts_stats['alljobs']=qtracts_avgs['alljobs']
qtracts_stats['med_rent']=qtracts_avgs['med_rent']

qtracts_stats.to_csv(output + 'tract_stats_notransit.csv')

qtracts_stats.head()

5751


,total_pop,pct_rent,pct_white,pct_hispanic,pct_black,pct_asian,pct_below200pov,hispanic_pov,black_pov,asian_pov,...,pct_medmf,pct_vacant,pct_since2000,pct_before1950,pct_bach_degree,pct_hh_kids,pct_unemp_civ,density,alljobs,med_rent
CBSA Title,,,,,,,,,,,,,,,,,,,,,
"Bakersfield, CA",779621.0,0.419590,0.367880,0.505378,0.052293,0.047355,0.465993,0.286247,0.350600,0.135807,...,0.057478,0.103602,0.220138,0.095558,0.164692,0.439436,0.117449,96.190758,94979.007299,1007.136364
"Chico, CA",223877.0,0.414642,0.734323,0.154116,0.014646,0.042421,0.429296,0.277212,0.370568,0.309698,...,0.069375,0.123143,0.139233,0.123727,0.261456,0.264312,0.107237,136.805414,28080.803922,988.333333
"Clearlake, CA",64076.0,0.368252,0.722595,0.188620,0.020741,0.013094,0.492120,0.351872,0.471756,0.405374,...,0.025342,0.264895,0.151545,0.090225,0.161568,0.266321,0.136766,50.997044,10090.533333,959.200000
"Crescent City, CA",27628.0,0.391295,0.630773,0.192775,0.022875,0.023527,0.460997,0.328247,0.447619,0.441536,...,0.027459,0.166255,0.158485,0.105245,0.149067,0.305623,0.104487,27.453042,5992.500000,765.142857
"El Centro, CA",178807.0,0.442271,0.119313,0.827037,0.023461,0.012455,0.503818,0.257607,0.289820,0.149450,...,0.094735,0.195164,0.265332,0.060661,0.139572,0.454891,0.170884,42.811596,25167.709677,837.766667
